In [1]:
import MySQLdb
import torch
import numpy as np
import matplotlib.pyplot as plt
import stock
import re
from IPython.display import display, Markdown
import ipywidgets as widgets
import xueqiu
import time
import kline

In [2]:
"""装入一个分类的所有数据并对齐，返回一个大的矩阵"""
def LoadCategoryCompanys(categoryName):
    db = MySQLdb.connect("localhost", "root", "789", "stock", charset='utf8',port=3307 )

    db.query("""select * from company_select where category='%s'"""%(categoryName))
    r = db.store_result()
    companys = r.fetch_row(r.num_rows())
    C,K,D = stock.loadKline('SZ:399001') #和深圳成指对齐
    KS = np.zeros((len(companys),len(K),5))
    i = 0
    for c in companys:
        c,k,d = stock.loadKline(c[1])
        KS[i] = stock.alignK(D,k,d)
        i+=1
    db.close()
    return companys,KS,D

# 行业手动初选

In [19]:
def CategorySelect(name):
    output = widgets.Output()
    def K(c,k,d):
        display(Markdown("""[%s %s ttm:%.2f pb:%.2f](https://xueqiu.com/S/%s)"""%(c[2],c[1],c[4],c[5],c[1])))
        butsel = widgets.Checkbox(value=True if c[-1]==1 else False,
            description='选择',
            disabled=False,
            indent=False)
        display(butsel,output)
        def onClick(event):
            value = 1 if event['new'] else 0
            db = MySQLdb.connect("localhost", "root", "789", "stock", charset='utf8',port=3307 )
            cursor = db.cursor()
            try:
                cursor.execute("""update company_select set manual=%d where code='%s'"""%(value,c[1]))
                db.commit()
            except Exception as e:
                print(e)
                db.rollback()            
            db.close()
            
        butsel.observe(onClick,names='value')
        
        ve = stock.volumeEnergyK(k,20)
        veKDJ = stock.kdj(ve,9)
        kline.Plote(k,date=d,config={
            "index":True,
            "figure":[[
                {
                    'data':veKDJ[:,2],
                    'color':'red',
                    'linewidth':2,
                    'hline':{
                        "y":0,
                        "color":'blue',
                        "linestyle":"-."
                    }
                },
                {
                    'data':VEKDJ[:,2],
                    'color':'fuchsia',
                    'linewidth':2,
                }
            ]]
        },companyInfo=c).show(len(k)-440,len(k),figsize=(36,15))    
    coms,ks,date = LoadCategoryCompanys(name)
    kvs = np.zeros((ks.shape[1]))
    for i in range(ks.shape[1]):
        kvs[i] = ks[:,i,0].sum()
    VE = stock.volumeEnergy(kvs,20)
    VEKDJ = stock.kdj(VE,9)
    for i in range(len(coms)):
        K(coms[i],ks[i],date)

In [20]:
CategorySelect('') #输入雪球分类名称